In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
folder = 'sp_matrix_tag_pop'
group_type = 'low'
group_dir_ = '../data/groups/' + group_type
group_info = 'group_info/' + group_type
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

In [2]:
df = pd.read_pickle(os.path.join(dir_, folder, group_info, file_name))
df[:5]

,uid,tid,count,tags,rating,pop,group_info_2,group_info_3,group_info_4,group_info_5
0,0,0,2,1,1.50,5,67.0,NaN,NaN,NaN
2,0,1,2,2,1.50,10,67.0,NaN,NaN,NaN
7,0,2,4,0,3.00,24,67.0,NaN,NaN,NaN
8,0,3,1,0,0.25,9,67.0,NaN,NaN,NaN
10,0,4,5,1,3.50,14,67.0,NaN,NaN,NaN


In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
group2 = {}
group3 = {}
group4 = {}
group5 = {}
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
    j = 0;
    for group in groups:
        for user in group:
            if len(group) == 2:
                group2[user] = j
            elif len(group) == 3:
                group3[user] = j
            elif len(group) == 4:
                group4[user] = j
            elif len(group) == 5:
                group5[user] = j
        j += 1

In [4]:
predictions2 = pd.read_pickle(os.path.join(dir_, folder, group_info, '2_topN_pred_' + file_name[:-3] + 'pkl'))
predictions3 = pd.read_pickle(os.path.join(dir_, folder, group_info, '3_topN_pred_' + file_name[:-3] + 'pkl'))
predictions4 = pd.read_pickle(os.path.join(dir_, folder, group_info, '4_topN_pred_' + file_name[:-3] + 'pkl'))
predictions5 = pd.read_pickle(os.path.join(dir_, folder, group_info, '5_topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [5]:
predictions2[:5]

,uid,tid,rating
0,0,2,0.076058
1,0,14,0.026010
2,0,15,0.156017
3,0,20,0.042927
4,0,21,0.026385


In [6]:
predictions2['gid'] = predictions2['uid'].map(group2)
predictions3['gid'] = predictions3['uid'].map(group3)
predictions4['gid'] = predictions4['uid'].map(group4)
predictions5['gid'] = predictions5['uid'].map(group5)

In [7]:
predictions5[:5]

,uid,tid,rating,gid
0,0,2,0.022639,NaN
1,0,14,0.035835,NaN
2,0,15,0.072896,NaN
3,0,20,0.032571,NaN
4,0,21,0.046337,NaN


In [8]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [9]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
len(groups_n[0][0]), len(groups_n[1][0]), len(groups_n[2][0]), len(groups_n[3][0])

(2, 3, 4, 5)

In [10]:
top_4_results = []
top_20_results = []
top_100_results = []

In [11]:
top_n_size = 100
top_n_lists_g = []
for groups in tqdm(groups_n):
    
    top_n_lists = []

    if len(groups[0]) == 2:
        predictions = predictions2
    elif len(groups[0]) == 3:
        predictions = predictions3
    elif len(groups[0]) == 4:
        predictions = predictions4
    else:
        predictions = predictions5

    for gid in range(len(groups)):
        
        rating_table = predictions[predictions['gid']==gid]
        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        print(rating_table)
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([100, 20, 4]):
    avg_precisions = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            high_rating = 0

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        break

            precision = high_rating / top_n_size
            precisions.append(precision)

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
    if top_n_size == 4:
        top_4_results.append(avg_precisions)
    elif top_n_size == 20:
        top_20_results.append(avg_precisions)
    elif top_n_size == 100:
        top_100_results.append(avg_precisions)

         uid    tid    rating  gid
4367583  116  14499  0.252529  0.0
4363067  116   8911  0.246224  0.0
3158293   84  14297  0.246099  0.0
3158269   84  14273  0.244413  0.0
3157069   84  13072  0.244049  0.0
...      ...    ...       ...  ...
4366160  116  12858  0.188690  0.0
4366351  116  13078  0.188653  0.0
3161389   84  17397  0.188590  0.0
3166518   84  22530  0.188582  0.0
3161578   84  17587  0.188554  0.0

[100 rows x 4 columns]
         uid    tid    rating  gid
5751203  153  12163  0.257307  1.0
5760151  153  21128  0.257201  1.0
5757249  153  18224  0.252198  1.0
5752518  153  13480  0.245898  1.0
261676     7   1360  0.244847  1.0
...      ...    ...       ...  ...
5755957  153  16927  0.188394  1.0
5761667  153  22644  0.188072  1.0
266601     7   7882  0.186809  1.0
261060     7    581  0.186529  1.0
260914     7    397  0.186457  1.0

[100 rows x 4 columns]
         uid    tid    rating  gid
4781024  127  13727  0.257641  2.0
4789686  127  22403  0.257535  2.0
4786862

         uid    tid    rating   gid
6674175  178   1306  0.257948  24.0
6682123  178  10329  0.257842  24.0
4818023  128  11761  0.256948  24.0
4826950  128  20759  0.256842  24.0
6679789  178   7597  0.252828  24.0
...      ...    ...       ...   ...
6675745  178   3087  0.194553  24.0
4822887  128  16657  0.194442  24.0
6678893  178   6538  0.194290  24.0
4823871  128  17647  0.193992  24.0
4822257  128  16022  0.193875  24.0

[100 rows x 4 columns]
         uid    tid    rating   gid
7016453  187  10917  0.257211  25.0
185746     5     55  0.256211  25.0
194769     5   9147  0.256105  25.0
191820     5   6190  0.251124  25.0
7008929  187   1199  0.245908  25.0
...      ...    ...       ...   ...
193080     5   7451  0.196336  25.0
191553     5   5920  0.196226  25.0
195016     5   9395  0.195906  25.0
186597     5    922  0.195500  25.0
7009228  187   1537  0.194055  25.0

[100 rows x 4 columns]
         uid    tid    rating   gid
3914536  104   8991  0.257319  26.0
3923040  104  17

         uid    tid    rating   gid
2701242   71  35114  0.257804  48.0
2521268   67   5655  0.257669  48.0
2548564   67  33190  0.257554  48.0
2518506   67   2852  0.252658  48.0
2549022   67  33650  0.246676  48.0
...      ...    ...       ...   ...
2517467   67   1787  0.195092  48.0
2553290   67  37934  0.194926  48.0
2516900   67   1204  0.194525  48.0
2554268   67  38912  0.194476  48.0
2552669   67  37313  0.194359  48.0

[100 rows x 4 columns]
         uid    tid    rating   gid
4941761  131  21906  0.257378  49.0
7007652  186  38915  0.257317  49.0
6977863  186   8684  0.256981  49.0
6974952  186   5741  0.251983  49.0
6969469  186     75  0.246226  49.0
...      ...    ...       ...   ...
4935695  131  15561  0.189195  49.0
4935589  131  15450  0.188949  49.0
6971495  186   2196  0.188891  49.0
4940379  131  20443  0.188744  49.0
4935054  131  14899  0.188695  49.0

[100 rows x 4 columns]
         uid    tid    rating   gid
6155864  163  35569  0.257871  50.0
5447912  144  33

         uid    tid    rating   gid
5009441  133  12971  0.256488  72.0
5018409  133  22417  0.256382  72.0
1660872   44   7805  0.252501  72.0
5015490  133  19320  0.251395  72.0
1656255   44   3101  0.246196  72.0
...      ...    ...       ...   ...
1660718   44   7651  0.194521  72.0
1659135   44   6044  0.194404  72.0
1659874   44   6792  0.194033  72.0
5015336  133  19159  0.193637  72.0
5011062  133  14668  0.193412  72.0

[100 rows x 4 columns]
        uid    tid    rating   gid
490527   13   9147  0.257793  73.0
483615   13   1360  0.244780  73.0
488604   13   6929  0.236263  73.0
489382   13   7828  0.232426  73.0
483064   13    779  0.230347  73.0
...     ...    ...       ...   ...
482917   13    624  0.167197  73.0
482936   13    643  0.167105  73.0
485394   13   3273  0.166956  73.0
491382   13  10149  0.166941  73.0
487969   13   6191  0.166909  73.0

[100 rows x 4 columns]
         uid    tid    rating   gid
1541015   40  37979  0.257819  74.0
1508239   40   4096  0.25129

         uid    tid    rating   gid
2293395   60  38704  0.257011  96.0
2263619   60   7718  0.256701  96.0
3882129  103  14729  0.252138  96.0
2260701   60   4689  0.251707  96.0
3876684  103   8947  0.246378  96.0
...      ...    ...       ...   ...
3877354  103   9640  0.191612  96.0
2260728   60   4716  0.191540  96.0
3877940  103  10247  0.191478  96.0
3877505  103   9796  0.191418  96.0
3876269  103   8518  0.191326  96.0

[100 rows x 4 columns]
         uid    tid    rating   gid
4477922  119  11886  0.246848  97.0
6896690  184   4377  0.244670  97.0
4477743  119  11705  0.244092  97.0
6930586  184  39080  0.243884  97.0
4483834  119  17942  0.236109  97.0
...      ...    ...       ...   ...
6930642  184  39136  0.175445  97.0
4480595  119  14622  0.175348  97.0
6896314  184   4001  0.175322  97.0
4480564  119  14591  0.175287  97.0
4478746  119  12734  0.175003  97.0

[100 rows x 4 columns]
         uid    tid    rating   gid
5860537  156   6691  0.257289  98.0
5869530  156  16

         uid    tid    rating   gid
2442401   64  38082  0.255711  17.0
1606466   42  29127  0.255546  17.0
993159    26  20795  0.254987  17.0
2412527   64   7388  0.254783  17.0
1603645   42  26298  0.250585  17.0
...      ...    ...       ...   ...
1598789   42  21403  0.203924  17.0
994369    26  22008  0.203825  17.0
1000917   26  28690  0.203376  17.0
994126    26  21762  0.203262  17.0
1598625   42  21238  0.202413  17.0

[100 rows x 4 columns]
         uid    tid    rating   gid
198035     5  12551  0.255832  18.0
4842061  128  35899  0.254223  18.0
195407     5   9788  0.250866  18.0
191037     5   5399  0.245434  18.0
190271     5   4631  0.245377  18.0
...      ...    ...       ...   ...
195434     5   9815  0.190747  18.0
190896     5   5258  0.190043  18.0
189899     5   4258  0.189992  18.0
4839184  128  33020  0.189500  18.0
194575     5   8953  0.189243  18.0

[100 rows x 4 columns]
         uid    tid    rating   gid
919833    24  24951  0.255780  19.0
928575    24  34

         uid    tid    rating   gid
3438028   91  28102  0.255614  39.0
5073622  135   1322  0.254637  39.0
5104896  135  33001  0.244238  39.0
5104067  135  32165  0.244181  39.0
5103564  135  31661  0.242142  39.0
...      ...    ...       ...   ...
5103650  135  31747  0.189041  39.0
5109601  135  37724  0.188919  39.0
5104898  135  33003  0.188660  39.0
5103808  135  31906  0.188531  39.0
5108761  135  36884  0.188293  39.0

[100 rows x 4 columns]
         uid    tid    rating   gid
3616490   96  14167  0.256171  40.0
3623898   96  21699  0.255923  40.0
1333465   35  19837  0.255348  40.0
1339400   35  25840  0.250147  40.0
3617650   96  15343  0.245522  40.0
...      ...    ...       ...   ...
3621390   96  19146  0.192673  40.0
1338417   35  24849  0.192643  40.0
1335080   35  21476  0.192533  40.0
3621226   96  18978  0.192361  40.0
1339246   35  25686  0.192031  40.0

[100 rows x 4 columns]
         uid    tid    rating   gid
3650953   97  10211  0.252615  41.0
3660044   97  19

         uid    tid    rating   gid
2608604   69  17413  0.253693  14.0
2617595   69  26448  0.253422  14.0
2614648   69  23482  0.248050  14.0
2609093   69  17906  0.243615  14.0
2609903   69  18718  0.243601  14.0
...      ...    ...       ...   ...
2619112   69  27965  0.181983  14.0
2614533   69  23366  0.181596  14.0
2610207   69  19023  0.181169  14.0
2611831   69  20653  0.180645  14.0
2608896   69  17709  0.180509  14.0

[100 rows x 4 columns]
         uid    tid    rating   gid
341279     9   9929  0.254587  15.0
2532146   67  16654  0.253553  15.0
2541073   67  25655  0.253282  15.0
534276    14  16802  0.252679  15.0
543314    14  26311  0.252408  15.0
...      ...    ...       ...   ...
2535094   67  19628  0.208992  15.0
545087    14  28084  0.208861  15.0
544775    14  27772  0.208828  15.0
2541855   67  26442  0.208585  15.0
341821     9  10484  0.208446  15.0

[100 rows x 4 columns]
         uid    tid    rating   gid
5414895  144      0  0.002297  16.0
6996087  186  27

         uid    tid    rating  gid
2424021   64  19360  0.254231  7.0
1002513   26  30286  0.253971  7.0
2432969   64  28493  0.253813  7.0
39125      1   1656  0.253010  7.0
48148      1  10740  0.252592  7.0
...      ...    ...       ...  ...
39387      1   1919  0.209312  7.0
49607      1  12199  0.209283  7.0
974000    26   1443  0.209191  7.0
2424317   64  19664  0.208438  7.0
2427022   64  22412  0.208402  7.0

[100 rows x 4 columns]
         uid    tid    rating  gid
697097    18  26034  0.254803  8.0
2193226   58  12879  0.254735  8.0
2201895   58  21810  0.254318  8.0
193873     5   8247  0.253444  8.0
202869     5  17385  0.253026  8.0
...      ...    ...       ...  ...
697834    18  26771  0.201117  8.0
2194018   58  13692  0.201063  8.0
201642     5  16158  0.201041  8.0
698210    18  27147  0.201026  8.0
2194429   58  14118  0.200971  8.0

[100 rows x 4 columns]
         uid    tid    rating  gid
3222075   86      0  0.002299  9.0
7420111  198  13511  0.002299  9.0
7420102

In [12]:
for i in top_4_results:
    print(i)
print('------------------')

for i in top_20_results:
    print(i)
print('------------------')

for i in top_100_results:
    print(i)

[0.025, 0.0, 0.057692307692307696, 0.03333333333333333]
------------------
[0.025, 0.0, 0.05961538461538462, 0.02666666666666667]
------------------
[0.0258, 0.0, 0.058076923076923075, 0.03066666666666667]


In [13]:
top_n_size = 100
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([100, 20, 4]):
    avg_precisions = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            high_rating = 0

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        break

            precision = high_rating / top_n_size
            precisions.append(precision)

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
    if top_n_size == 4:
        top_4_results.append(avg_precisions)
    elif top_n_size == 20:
        top_20_results.append(avg_precisions)
    elif top_n_size == 100:
        top_100_results.append(avg_precisions)

In [14]:
for i in top_4_results:
    print(i)
print('------------------')

for i in top_20_results:
    print(i)
print('------------------')

for i in top_100_results:
    print(i)

[0.025, 0.0, 0.057692307692307696, 0.03333333333333333]
[0.0325, 0.005555555555555556, 0.038461538461538464, 0.0]
------------------
[0.025, 0.0, 0.05961538461538462, 0.02666666666666667]
[0.030999999999999993, 0.017777777777777785, 0.023076923076923075, 0.016666666666666666]
------------------
[0.0258, 0.0, 0.058076923076923075, 0.03066666666666667]
[0.026099999999999977, 0.02200000000000001, 0.027307692307692318, 0.022000000000000006]
